In [220]:
# Chargement des fichiers Excel depuis les dossiers spécifiques (peu importe leur nom et format)
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime, date
from pathlib import Path
from currency_converter import CurrencyConverter

# Extensions supportées pour les fichiers Excel et CSV
EXCEL_EXTENSIONS = ['.xlsx', '.xlsm', '.xls', '.xlsb', '.xltx', '.xltm', '.xlt', '.csv']

# Fonction pour trouver le fichier Excel/CSV dans un dossier
def find_excel_file(directory):
    """Trouve le premier fichier Excel ou CSV dans le dossier spécifié"""
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Dossier '{directory}' introuvable")
    
    # Chercher tous les fichiers avec les extensions supportées
    excel_files = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path):
            file_ext = os.path.splitext(file)[1].lower()
            if file_ext in EXCEL_EXTENSIONS:
                excel_files.append(file)
    
    if len(excel_files) == 0:
        raise FileNotFoundError(f"Aucun fichier Excel/CSV trouvé dans '{directory}'")
    if len(excel_files) > 1:
        raise ValueError(f"Plusieurs fichiers Excel/CSV trouvés dans '{directory}': {excel_files}")
    
    return os.path.join(directory, excel_files[0])

# Fonction pour charger un fichier Excel/CSV selon son extension
def load_excel_file(file_path, header=0, sheet_name=None):
    """Charge un fichier Excel ou CSV selon son extension
    
    Parameters:
    -----------
    file_path : str
        Chemin vers le fichier à charger
    header : int, default=0
        Numéro de ligne à utiliser comme en-têtes (0 = première ligne)
    sheet_name : str ou int, optional
        Nom ou index de la feuille à charger (uniquement pour Excel, pas CSV)
        Si None, ne passe pas le paramètre pour charger la première feuille par défaut
    """
    file_ext = os.path.splitext(file_path)[1].lower()
    
    if file_ext == '.csv':
        return pd.read_csv(file_path, header=header)
    else:
        # Ne pas passer sheet_name si c'est None pour éviter les problèmes avec les fichiers à une seule feuille
        if sheet_name is None:
            return pd.read_excel(file_path, header=header)
        else:
            return pd.read_excel(file_path, header=header, sheet_name=sheet_name)

# Initialisation du convertisseur de devises
converter = CurrencyConverter()

# Chargement des fichiers depuis les dossiers spécifiques
tab_source_path = find_excel_file('data/Extract LLM de FT')
spire_recap_path = find_excel_file('data/Spire Recap')

# Spécifier le nom de la feuille pour Spire Recap (None = première feuille par défaut)
SPIRE_SHEET_NAME = None  # Modifier ici pour spécifier une feuille (ex: "Feuille1", 0 pour la première, etc.)

ini = load_excel_file(tab_source_path)
df_spire = load_excel_file(spire_recap_path, header=1, sheet_name=SPIRE_SHEET_NAME)  # En-têtes à la ligne 2 (index 1)

# Normaliser les noms de colonnes de df_spire (supprimer espaces, mettre en majuscules)
# Gérer le cas où les colonnes peuvent être de différents types
df_spire.columns = [str(col).strip().upper() for col in df_spire.columns]

tableau_final = pd.DataFrame()


ℹ API non disponible - utilisation uniquement du Excel de secours
ℹ Aucun fichier Excel de secours trouvé


In [221]:
# Définition des dictionnaires de correspondance (dealers, collatéraux)
dealer_name = {
    "BNP Paribas": "BNPP",
}

collat_name = {
    "Republic of Italy": "BTP",
}


In [ ]:
# Fonctions utilitaires pour conversions de types robustes
def safe_str(value):
    """Convertit une valeur en string de manière sûre"""
    if pd.isna(value):
        return ""
    return str(value).strip()

def safe_float(value, default=np.nan):
    """Convertit une valeur en float de manière sûre"""
    if pd.isna(value):
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

def safe_int(value, default=None):
    """Convertit une valeur en int de manière sûre"""
    if pd.isna(value):
        return default
    try:
        return int(float(value))
    except (ValueError, TypeError):
        return default

def safe_date(value, default=None):
    """Convertit une valeur en date de manière sûre"""
    if pd.isna(value):
        return default
    if isinstance(value, (date, datetime)):
        return value.date() if isinstance(value, datetime) else value
    try:
        if isinstance(value, str):
            return pd.to_datetime(value).date()
        return pd.to_datetime(value).date()
    except (ValueError, TypeError):
        return default

def safe_compare(value1, value2):
    """Compare deux valeurs en les convertissant en string si nécessaire"""
    if pd.isna(value1) or pd.isna(value2):
        return False
    try:
        # Essayer comparaison directe
        return value1 == value2
    except:
        # Si échec, comparer en string
        return safe_str(value1) == safe_str(value2)

# Définition des fonctions utilitaires (formatage dates, parsing formules de taux, etc.)
def format_date_dd_mon_yy(date_value):
    """Formate une date au format DD-Mon-YY, gère tous les types"""
    if pd.isna(date_value):
        return ""
    try:
        date_obj = safe_date(date_value)
        if date_obj is None:
            return ""
        if isinstance(date_obj, date):
            return date_obj.strftime("%d-%b-%y")
        return pd.to_datetime(date_obj).strftime("%d-%b-%y")
    except:
        return ""

def format_percentage(value):
    """Formate une valeur en pourcentage, gère tous les types"""
    if pd.isna(value):
        return ""
    try:
        percentage = safe_float(value) * 100
        if pd.isna(percentage):
            return ""
        return f"{percentage:.2f}%"
    except:
        return ""

def parse_rate_formula(formula_str):
    """Parse une formule de taux, gère tous les types"""
    if pd.isna(formula_str):
        return None
    formula = safe_str(formula_str)
    result = {'rate_base': '', 'spread': '', 'floor': '', 'cap': ''}
    
    if 'Floor at' in formula or 'Cap at' in formula:
        rate_match = re.search(r'([A-Z0-9\s\-]+?)\s*\+\s*([\d.]+)%', formula)
        if rate_match:
            result['rate_base'] = rate_match.group(1).strip()
            result['spread'] = rate_match.group(2)
        floor_match = re.search(r'Floor\s+at\s+([\d.]+)%', formula, re.IGNORECASE)
        if floor_match:
            result['floor'] = floor_match.group(1)
        cap_match = re.search(r'Cap\s+at\s+([\d.]+)%', formula, re.IGNORECASE)
        if cap_match:
            result['cap'] = cap_match.group(1)
    elif 'MIN' in formula.upper() and 'MAX' in formula.upper():
        min_match = re.search(r'MIN\s*\[?\s*([\d.]+)%', formula, re.IGNORECASE)
        if min_match:
            result['cap'] = min_match.group(1)
        max_match = re.search(r'MAX\s*\[?\s*([\d.]+)%\s*[;,]?\s*([^)]+)\+?\s*([\d.]+)%?', formula, re.IGNORECASE)
        if max_match:
            result['floor'] = max_match.group(1)
            rate_part = max_match.group(2).strip() if len(max_match.groups()) >= 2 else ''
            spread_part = max_match.group(3) if len(max_match.groups()) >= 3 else ''
            rate_base_match = re.search(r'([A-Z0-9\s\-]+?)(?:\s*\+\s*[\d.]+%?)?$', rate_part)
            if rate_base_match:
                result['rate_base'] = rate_base_match.group(1).strip()
            if spread_part:
                result['spread'] = spread_part
    
    if not result['rate_base']:
        rate_base_match = re.search(r'([A-Z]{2,}[0-9A-Z\s\-]*?)(?:\s*\+\s*[\d.]+%?)?', formula)
        if rate_base_match:
            result['rate_base'] = rate_base_match.group(1).strip()
    
    if 'EURIBOR' in result['rate_base'].upper():
        result['rate_base'] = 'EUR6M'
    
    return result

def format_floating_coupon(floating_note):
    if pd.isna(floating_note):
        return ""
    parsed = parse_rate_formula(floating_note)
    if not parsed:
        return ""
    rate_base = parsed['rate_base'] or 'Rate'
    spread = parsed['spread'] or '0'
    floor = parsed['floor'] or '0'
    cap = parsed['cap'] or ''
    if cap:
        return f"Y1 - End: Min({cap}% ; Max({rate_base} + {spread}% ; {floor}%))"
    else:
        return f"Y1 - End: Max({rate_base} + {spread}% ; {floor}%)"

def format_variable_linked_coupon(fixed_note, variable_note):
    """Formate un coupon variable-linked, gère tous les types"""
    if pd.isna(fixed_note) or pd.isna(variable_note):
        return ""
    parsed = parse_rate_formula(variable_note)
    if not parsed:
        return ""
    rate_base = parsed['rate_base'] or 'Rate'
    spread = parsed['spread'] or '0'
    floor = parsed['floor'] or '0'
    cap = parsed['cap'] or ''
    fixed_str = safe_str(fixed_note)
    if cap:
        return f"Y1 - Y2: {fixed_str}% p.a.\nY2 - End: {fixed_str}% p.a. or Min({cap}% ; Max({rate_base} + {spread}% ; {floor}%)) p.a."
    else:
        return f"Y1 - Y2: {fixed_str}% p.a.\nY2 - End: {fixed_str}% p.a. or Max({rate_base} + {spread}% ; {floor}%) p.a."

def process_pipe_separated(value, formatter=None):
    """Traite les valeurs séparées par |, gère tous les types"""
    if pd.isna(value):
        return ""
    value_str = safe_str(value)
    if '|' not in value_str:
        if formatter:
            return formatter(value_str)
        return value_str
    parts = [part.strip() for part in value_str.split('|')]
    if formatter:
        formatted_parts = [formatter(part) for part in parts]
        return " | ".join(formatted_parts)
    else:
        return " | ".join(parts)

def format_number_with_spaces(value):
    """Formate un nombre avec séparateur de milliers (espaces), gère tous les types"""
    if pd.isna(value):
        return ""
    try:
        num = safe_int(value)
        if num is None:
            return ""
        return f"{num:,}".replace(",", " ")
    except:
        return ""


In [223]:
# Création colonne Dealer : mapping depuis ini avec remplacement via dictionnaire
tableau_final['Dealer'] = ini['Dealer'].map(lambda x: dealer_name.get(safe_str(x), safe_str(x)) if pd.notna(x) else "")


In [224]:
# Création colonne ISIN (all) : copie directe depuis ini
tableau_final['ISIN (all)'] = ini['ISIN']


In [ ]:
# Création colonne N° Issuance : recherche dans df_spire via ISIN pour récupérer l'ID
def get_issuance_number(isin):
    print(f"[DEBUG N° Issuance] ISIN reçu: {isin} (type: {type(isin)})")
    
    if pd.isna(isin):
        print("[DEBUG N° Issuance] ISIN est NaN, retourne vide")
        return ""
    
    # Chercher la colonne ISIN (peut avoir des variations)
    print(f"[DEBUG N° Issuance] Colonnes disponibles dans df_spire: {list(df_spire.columns)}")
    isin_col = None
    for col in df_spire.columns:
        col_str = safe_str(col).upper()
        print(f"[DEBUG N° Issuance] Vérification colonne: '{col}' -> '{col_str}'")
        if 'ISIN' in col_str:
            isin_col = col
            print(f"[DEBUG N° Issuance] Colonne ISIN trouvée: '{isin_col}'")
            break
    
    if isin_col is None:
        print("[DEBUG N° Issuance] ERREUR: Aucune colonne ISIN trouvée dans df_spire")
        return ""
    
    # Comparaison robuste avec conversion en string si nécessaire
    isin_str = safe_str(isin)
    print(f"[DEBUG N° Issuance] ISIN converti en string: '{isin_str}'")
    print(f"[DEBUG N° Issuance] Premières valeurs de la colonne ISIN: {df_spire[isin_col].head().tolist()}")
    
    matching_rows = df_spire[df_spire[isin_col].apply(lambda x: safe_compare(x, isin_str))]
    print(f"[DEBUG N° Issuance] Nombre de lignes correspondantes: {len(matching_rows)}")
    
    if not matching_rows.empty:
        print(f"[DEBUG N° Issuance] Ligne(s) trouvée(s), recherche de la colonne ID...")
        # Chercher la colonne ID (peut avoir des variations)
        id_col = None
        for col in df_spire.columns:
            col_str = safe_str(col).upper().strip()
            print(f"[DEBUG N° Issuance] Vérification colonne ID: '{col}' -> '{col_str}'")
            if col_str == 'ID' or 'N°ISSUANCE' in col_str or 'N° ISSUANCE' in col_str or 'N°ISSUANCE' in col_str.replace(' ', ''):
                id_col = col
                print(f"[DEBUG N° Issuance] Colonne ID trouvée: '{id_col}'")
                break
        
        if id_col is None:
            print("[DEBUG N° Issuance] ERREUR: Aucune colonne ID trouvée dans df_spire")
            return ""
        
        id_value = matching_rows.iloc[0][id_col]
        print(f"[DEBUG N° Issuance] Valeur ID trouvée: {id_value} (type: {type(id_value)})")
        result = safe_str(id_value) if pd.notna(id_value) else ""
        print(f"[DEBUG N° Issuance] Résultat final: '{result}'")
        return result
    else:
        print(f"[DEBUG N° Issuance] Aucune correspondance trouvée pour ISIN '{isin_str}'")
    return ""

tableau_final['N° Issuance'] = ini['ISIN'].apply(get_issuance_number)


In [ ]:
# Création colonne Creation Date : recherche dans df_spire via ISIN et formatage date DD-Mon-YY
def get_creation_date(isin):
    print(f"[DEBUG Creation Date] ISIN reçu: {isin} (type: {type(isin)})")
    
    if pd.isna(isin):
        print("[DEBUG Creation Date] ISIN est NaN, retourne vide")
        return ""
    
    # Chercher la colonne ISIN (peut avoir des variations)
    print(f"[DEBUG Creation Date] Colonnes disponibles dans df_spire: {list(df_spire.columns)}")
    isin_col = None
    for col in df_spire.columns:
        col_str = safe_str(col).upper()
        print(f"[DEBUG Creation Date] Vérification colonne: '{col}' -> '{col_str}'")
        if 'ISIN' in col_str:
            isin_col = col
            print(f"[DEBUG Creation Date] Colonne ISIN trouvée: '{isin_col}'")
            break
    
    if isin_col is None:
        print("[DEBUG Creation Date] ERREUR: Aucune colonne ISIN trouvée dans df_spire")
        return ""
    
    # Comparaison robuste avec conversion en string si nécessaire
    isin_str = safe_str(isin)
    print(f"[DEBUG Creation Date] ISIN converti en string: '{isin_str}'")
    print(f"[DEBUG Creation Date] Premières valeurs de la colonne ISIN: {df_spire[isin_col].head().tolist()}")
    
    matching_rows = df_spire[df_spire[isin_col].apply(lambda x: safe_compare(x, isin_str))]
    print(f"[DEBUG Creation Date] Nombre de lignes correspondantes: {len(matching_rows)}")
    
    if not matching_rows.empty:
        print(f"[DEBUG Creation Date] Ligne(s) trouvée(s), recherche de la colonne Creation Date...")
        # Chercher la colonne Creation Date (peut avoir des variations)
        creation_date_col = None
        for col in df_spire.columns:
            col_str = safe_str(col).upper()
            print(f"[DEBUG Creation Date] Vérification colonne Creation Date: '{col}' -> '{col_str}'")
            if 'CREATION' in col_str and 'DATE' in col_str:
                creation_date_col = col
                print(f"[DEBUG Creation Date] Colonne Creation Date trouvée: '{creation_date_col}'")
                break
        
        if creation_date_col is None:
            print("[DEBUG Creation Date] ERREUR: Aucune colonne Creation Date trouvée dans df_spire")
            return ""
        
        creation_date = matching_rows.iloc[0][creation_date_col]
        print(f"[DEBUG Creation Date] Valeur Creation Date trouvée: {creation_date} (type: {type(creation_date)})")
        if pd.notna(creation_date):
            formatted_date = format_date_dd_mon_yy(creation_date)
            print(f"[DEBUG Creation Date] Date formatée: '{formatted_date}'")
            return formatted_date
        else:
            print("[DEBUG Creation Date] Creation Date est NaN")
    else:
        print(f"[DEBUG Creation Date] Aucune correspondance trouvée pour ISIN '{isin_str}'")
    return ""

tableau_final['Creation Date'] = ini['ISIN'].apply(get_creation_date)


In [227]:
# Création colonne Maturity : formatage de la date d'échéance au format DD-Mon-YY
tableau_final['Maturity'] = ini['Maturity Date'].apply(format_date_dd_mon_yy)


In [228]:
# Création colonne Currency : copie directe depuis ini
tableau_final['Currency'] = ini['Currency']


In [ ]:
# Création colonne Equiv EUR : conversion du nominal en EUR avec formatage séparateur de milliers
def convert_to_eur_value(row):
    """Convertit un montant vers EUR en utilisant CurrencyConverter"""
    amount = row['Nominal']
    currency = row['Currency']
    target_date = row.get('Maturity Date', date.today())
    
    if pd.isna(amount) or pd.isna(currency):
        return np.nan
    
    amount_float = safe_float(amount)
    if pd.isna(amount_float):
        return np.nan
    
    currency_str = safe_str(currency).upper()
    if not currency_str:
        return np.nan
    
    if currency_str == 'EUR':
        return amount_float
    
    target_date_obj = safe_date(target_date, date.today())
    
    try:
        converted = converter.convert(amount_float, currency_str, 'EUR', target_date_obj)
        return converted if converted is not None else np.nan
    except:
        return np.nan

tableau_final['Equiv EUR'] = ini.apply(lambda row: format_number_with_spaces(convert_to_eur_value(row)), axis=1)


In [230]:
# Création colonne Issue Price : conversion en pourcentage avec 2 décimales
tableau_final['Issue Price'] = ini['Issue Price'].apply(format_percentage)


In [231]:
# Création colonne Collat Name : mapping via dictionnaire et ajout "I/L" si inflation linked
def process_collat_name(row):
    collat_name_val = row.get('Collat Name', '')
    inflation_linked = row.get('Inflation Linked? (Collat)', '')
    if pd.isna(collat_name_val):
        return ""
    collat_str = safe_str(collat_name_val)
    if '|' in collat_str:
        parts = [p.strip() for p in collat_str.split('|')]
        mapped_parts = [collat_name.get(part, part) for part in parts]
        return " | ".join(mapped_parts)
    else:
        mapped_name = collat_name.get(collat_str, collat_str)
        inflation_str = safe_str(inflation_linked).upper()
        if inflation_str == 'YES':
            mapped_name += " I/L"
        return mapped_name

tableau_final['Collat Name'] = ini.apply(process_collat_name, axis=1)


In [232]:
# Création colonne Collat ISIN : gestion des valeurs séparées par "|"
tableau_final['Collat ISIN'] = ini['Collat ISIN'].apply(lambda x: process_pipe_separated(x))


In [233]:
# Création colonne Collat CCY : gestion des valeurs séparées par "|"
tableau_final['Collat CCY'] = ini['Collat CCY'].apply(lambda x: process_pipe_separated(x))


In [ ]:
# Création colonne Levrage : calcul du levier (collat en EUR / Equiv EUR) en pourcentage
def calculate_leverage_multiple(collat_amount, collat_ccy, nominal, currency, maturity_date=None):
    """Calcule le levier, gère tous les types de données"""
    if pd.isna(collat_amount) or pd.isna(collat_ccy) or pd.isna(nominal) or pd.isna(currency):
        return ""
    if maturity_date is None:
        maturity_date = date.today()
    else:
        maturity_date = safe_date(maturity_date, date.today())
    
    collat_amount_str = safe_str(collat_amount)
    collat_ccy_str = safe_str(collat_ccy)
    
    # Convertir le nominal en EUR
    nominal_float = safe_float(nominal)
    currency_str = safe_str(currency).upper()
    if pd.isna(nominal_float) or not currency_str:
        return ""
    
    if currency_str == 'EUR':
        equiv_eur = nominal_float
    else:
        try:
            equiv_eur = converter.convert(nominal_float, currency_str, 'EUR', maturity_date)
            if equiv_eur is None or pd.isna(equiv_eur):
                return ""
        except:
            return ""
    
    if equiv_eur == 0:
        return ""
    
    if '|' not in collat_amount_str and '|' not in collat_ccy_str:
        # Cas simple : une seule valeur
        collat_amount_float = safe_float(collat_amount)
        collat_ccy_str_single = safe_str(collat_ccy).upper()
        if pd.isna(collat_amount_float) or not collat_ccy_str_single:
            return ""
        
        if collat_ccy_str_single == 'EUR':
            collat_eur = collat_amount_float
        else:
            try:
                collat_eur = converter.convert(collat_amount_float, collat_ccy_str_single, 'EUR', maturity_date)
                if collat_eur is None or pd.isna(collat_eur):
                    return ""
            except:
                return ""
        
        leverage = (collat_eur / equiv_eur) * 100
        return f"{leverage:.2f}%"
    
    # Cas avec plusieurs valeurs séparées par |
    amounts = [a.strip() for a in collat_amount_str.split('|')]
    ccies = [c.strip() for c in collat_ccy_str.split('|')]
    leverages = []
    for i, amount in enumerate(amounts):
        if i < len(ccies):
            ccy = safe_str(ccies[i]).upper()
            if not ccy:
                continue
            try:
                amount_float = safe_float(amount)
                if pd.isna(amount_float):
                    continue
                
                if ccy == 'EUR':
                    collat_eur = amount_float
                else:
                    collat_eur = converter.convert(amount_float, ccy, 'EUR', maturity_date)
                    if collat_eur is None or pd.isna(collat_eur):
                        continue
                
                leverage = (collat_eur / equiv_eur) * 100
                leverages.append(f"{leverage:.2f}%")
            except:
                pass
    return " | ".join(leverages) if leverages else ""

tableau_final['Levrage'] = ini.apply(lambda row: calculate_leverage_multiple(
    row.get('Collat Amount', ''),
    row.get('Collat CCY', ''),
    row['Nominal'],
    row['Currency'],
    row.get('Maturity Date', date.today())
), axis=1)


In [235]:
# Création colonne Coupon : formatage selon le type (Fixed, Floating, Variable-linked, Zero Coupon)
def process_coupon(row):
    """Traite les coupons, gère tous les types de données"""
    interest_basis = safe_str(row.get('Interest Basis', '')).strip()
    if interest_basis == 'Fixed':
        fixed_note = row.get('Fixed Note', '')
        if pd.notna(fixed_note):
            return f"Y1 - End: {safe_str(fixed_note)}% p.a."
        return ""
    elif interest_basis == 'Fixed, Variable-linked':
        fixed_note = row.get('Fixed Note', '')
        variable_note = row.get('Variable-linked Note', '')
        return format_variable_linked_coupon(fixed_note, variable_note)
    elif interest_basis == 'Floating':
        floating_note = row.get('Floating Note', '')
        return format_floating_coupon(floating_note)
    elif interest_basis == 'Zero Coupon':
        issue_price = row.get('Issue Price', np.nan)
        maturity_date = row.get('Maturity Date', np.nan)
        issue_date = row.get('Issue Date', np.nan)
        if pd.notna(issue_price) and pd.notna(maturity_date) and pd.notna(issue_date):
            try:
                maturity = safe_date(maturity_date)
                issue = safe_date(issue_date)
                if maturity is None or issue is None:
                    return ""
                years_diff = (maturity - issue).days / 365.25
                issue_price_float = safe_float(issue_price)
                if not pd.isna(issue_price_float) and issue_price_float > 0 and years_diff > 0:
                    irr = -1 + (1 / issue_price_float) ** (1 / years_diff)
                    irr_percent = irr * 100
                    return f"ZC - {irr_percent:.2f}% IRR"
            except Exception as e:
                pass
        return ""
    return ""

tableau_final['Coupon'] = ini.apply(process_coupon, axis=1)


In [236]:
# Création colonne Final Redemption : conversion en pourcentage si nécessaire
def process_final_redemption(value):
    """Traite Final Redemption, gère tous les types de données"""
    if pd.isna(value):
        return ""
    value_str = safe_str(value)
    if '%' in value_str:
        return value_str
    try:
        num_value = safe_float(value)
        if not pd.isna(num_value):
            if 0 <= num_value <= 1:
                return f"{num_value * 100:.2f}%"
            elif 1 < num_value <= 100:
                return f"{num_value:.2f}%"
    except:
        pass
    return ""

tableau_final['Final Redemption'] = ini['Final Redemption'].apply(process_final_redemption)


In [237]:
# Création colonne Other comments : Issuer Call et Issuer Switch Option si présents
def process_other_comments(row):
    """Traite Other comments, gère tous les types de données"""
    comments = []
    issuer_call_date = row.get('Issuer Call Redemption Date', '')
    issuer_call_amount = row.get('Issuer Call Redemption Amount', '')
    
    issuer_call_date_str = safe_str(issuer_call_date).upper()
    if pd.notna(issuer_call_date) and issuer_call_date_str != 'N/A':
        date_str = safe_str(issuer_call_date)
        issuer_call_amount_str = safe_str(issuer_call_amount).upper()
        if pd.notna(issuer_call_amount) and issuer_call_amount_str != 'N/A':
            try:
                # Convertir le montant en pourcentage entier
                amount_value = safe_float(issuer_call_amount)
                if not pd.isna(amount_value):
                    if 0 <= amount_value <= 1:
                        amount_str = f"{safe_int(amount_value * 100)}%"
                    elif 1 < amount_value <= 100:
                        amount_str = f"{safe_int(amount_value)}%"
                    else:
                        amount_str = safe_str(issuer_call_amount)
                else:
                    amount_str = safe_str(issuer_call_amount)
            except:
                amount_str = safe_str(issuer_call_amount)
            comments.append(f"Issuer Call {date_str} @{amount_str}")
        else:
            comments.append(f"Issuer Call {date_str}")
    type_of_coupon = safe_str(row.get('Type of coupon', ''))
    if pd.notna(row.get('Type of coupon', '')) and ',' in type_of_coupon:
        comments.append("Issuer Switch Option YYYY-MM-DD")
    return " ".join(comments) if comments else ""

tableau_final['Other comments'] = ini.apply(process_other_comments, axis=1)


In [238]:
# Vérification du nombre de colonnes (doit être 15) et affichage du tableau final
if len(tableau_final.columns) != 15:
    import warnings
    warnings.warn(f"ATTENTION: Le tableau final contient {len(tableau_final.columns)} colonnes au lieu de 15 attendues. Colonnes: {list(tableau_final.columns)}")
tableau_final.head()


,Dealer,ISIN (all),N° Issuance,Creation Date,Maturity,Currency,Equiv EUR,Issue Price,Collat Name,Collat ISIN,Collat CCY,Levrage,Coupon,Final Redemption,Other comments
0,BNPP,XS2030639145,2654,17-Dec-24,15-May-56,EUR,25 000 000,100.00%,BTP I/L,IT0005647273,EUR,98.00%,Y1 - End: Min(6.00% ; Max(EUR6M + 3.14% ; 0.00%)),100%,
1,HSBC Bank plc,XS2135238659,2543,18-Nov-23,25-Jul-53,EUR,465 000 000,100.00%,Republic of France I/L,FR0014001881,EUR,13.96%,Y1 - End: 4.88% p.a.,100%,Issuer Call 2042-09-15 @100%
2,J.P. Morgan SE,XS2041123880,2020,25-May-35,15-Sep-42,EUR,123 480 766,75.33%,Basket of Gov,ES0000012E51 | ES0000012932 | ES0000012L60 | E...,EUR | EUR | EUR | EUR,,ZC - 1.69% IRR,,
3,Nomura Financial Products Europe GmbH,XS3205809513,4343,18-Dec-24,15-Sep-42,EUR,36 481 250,68.53%,The Republic of Italy I/L,IT0005547812,EUR,86.62%,ZC - 2.26% IRR,,


In [ ]:
# Cellule de test pour diagnostiquer les problèmes de conversion de devises
print("=" * 80)
print("TESTS DE DIAGNOSTIC DES CONVERSIONS DE DEVISES")
print("=" * 80)

# Test 1: Vérification de l'initialisation du CurrencyConverter
print("\n[TEST 1] Vérification de l'initialisation du CurrencyConverter")
print(f"Type de converter: {type(converter)}")
print(f"converter existe: {converter is not None}")

# Test 2: Vérification des données source
print("\n[TEST 2] Vérification des données source")
if 'ini' in locals() and not ini.empty:
    print(f"Nombre de lignes dans ini: {len(ini)}")
    print(f"Colonnes disponibles dans ini: {list(ini.columns)}")
    
    # Vérifier les colonnes nécessaires
    required_cols = ['Nominal', 'Currency', 'Maturity Date']
    for col in required_cols:
        if col in ini.columns:
            print(f"  ✓ Colonne '{col}' trouvée")
            # Afficher quelques exemples de valeurs
            sample_values = ini[col].dropna().head(3).tolist()
            print(f"    Exemples de valeurs: {sample_values}")
            print(f"    Types de données: {[type(v).__name__ for v in sample_values]}")
        else:
            print(f"  ✗ Colonne '{col}' MANQUANTE")
else:
    print("  ✗ DataFrame 'ini' non trouvé ou vide")

# Test 3: Test de conversion simple
print("\n[TEST 3] Test de conversion simple avec CurrencyConverter")
test_cases = [
    (1000, 'USD', 'EUR', date.today()),
    (1000, 'GBP', 'EUR', date.today()),
    (1000, 'EUR', 'EUR', date.today()),
]

for amount, from_curr, to_curr, test_date in test_cases:
    print(f"\n  Test: {amount} {from_curr} -> {to_curr} (date: {test_date})")
    try:
        result = converter.convert(amount, from_curr, to_curr, test_date)
        print(f"    ✓ Résultat: {result} (type: {type(result).__name__})")
        if result is None:
            print(f"    ⚠ ATTENTION: Résultat None")
        if pd.isna(result):
            print(f"    ⚠ ATTENTION: Résultat NaN")
    except Exception as e:
        print(f"    ✗ ERREUR: {type(e).__name__}: {str(e)}")
        import traceback
        print(f"    Traceback: {traceback.format_exc()}")

# Test 4: Test avec les données réelles du tableau ini
print("\n[TEST 4] Test avec les données réelles du tableau ini")
if 'ini' in locals() and not ini.empty and 'Nominal' in ini.columns and 'Currency' in ini.columns:
    # Prendre les 3 premières lignes non vides
    test_rows = ini[['Nominal', 'Currency', 'Maturity Date']].dropna(subset=['Nominal', 'Currency']).head(3)
    
    if len(test_rows) > 0:
        print(f"  Test sur {len(test_rows)} lignes réelles:")
        for idx, row in test_rows.iterrows():
            print(f"\n  Ligne {idx}:")
            nominal = row['Nominal']
            currency = row['Currency']
            maturity = row.get('Maturity Date', date.today())
            
            print(f"    Nominal: {nominal} (type: {type(nominal).__name__})")
            print(f"    Currency: {currency} (type: {type(currency).__name__})")
            print(f"    Maturity Date: {maturity} (type: {type(maturity).__name__})")
            
            # Test avec safe_float
            try:
                nominal_float = safe_float(nominal)
                print(f"    safe_float(nominal): {nominal_float} (type: {type(nominal_float).__name__})")
            except Exception as e:
                print(f"    ✗ ERREUR safe_float: {e}")
                nominal_float = None
            
            # Test avec safe_str
            try:
                currency_str = safe_str(currency).upper()
                print(f"    safe_str(currency).upper(): '{currency_str}'")
            except Exception as e:
                print(f"    ✗ ERREUR safe_str: {e}")
                currency_str = None
            
            # Test avec safe_date
            try:
                maturity_date_obj = safe_date(maturity, date.today())
                print(f"    safe_date(maturity): {maturity_date_obj} (type: {type(maturity_date_obj).__name__})")
            except Exception as e:
                print(f"    ✗ ERREUR safe_date: {e}")
                maturity_date_obj = date.today()
            
            # Test de conversion
            if nominal_float is not None and currency_str:
                try:
                    if currency_str == 'EUR':
                        result = nominal_float
                        print(f"    Conversion (EUR): {result}")
                    else:
                        result = converter.convert(nominal_float, currency_str, 'EUR', maturity_date_obj)
                        print(f"    Conversion ({currency_str} -> EUR): {result} (type: {type(result).__name__})")
                        if result is None:
                            print(f"    ⚠ ATTENTION: Résultat None")
                        if pd.isna(result):
                            print(f"    ⚠ ATTENTION: Résultat NaN")
                except Exception as e:
                    print(f"    ✗ ERREUR conversion: {type(e).__name__}: {str(e)}")
                    import traceback
                    print(f"    Traceback: {traceback.format_exc()}")
    else:
        print("  ✗ Aucune ligne avec Nominal et Currency valides trouvée")
else:
    print("  ✗ Impossible de tester avec les données réelles (colonnes manquantes)")

# Test 5: Test de la fonction convert_to_eur_value (si elle existe)
print("\n[TEST 5] Test de la fonction convert_to_eur_value")
if 'convert_to_eur_value' in locals():
    print("  ✓ Fonction convert_to_eur_value trouvée")
    if 'ini' in locals() and not ini.empty:
        try:
            test_row = ini.iloc[0]
            result = convert_to_eur_value(test_row)
            print(f"  Test sur première ligne: {result} (type: {type(result).__name__})")
        except Exception as e:
            print(f"  ✗ ERREUR lors du test: {type(e).__name__}: {str(e)}")
            import traceback
            print(f"  Traceback: {traceback.format_exc()}")
else:
    print("  ✗ Fonction convert_to_eur_value non trouvée")

# Test 6: Test de calculate_leverage_multiple
print("\n[TEST 6] Test de la fonction calculate_leverage_multiple")
if 'calculate_leverage_multiple' in locals():
    print("  ✓ Fonction calculate_leverage_multiple trouvée")
    if 'ini' in locals() and not ini.empty:
        required_cols_leverage = ['Collat Amount', 'Collat CCY', 'Nominal', 'Currency', 'Maturity Date']
        if all(col in ini.columns for col in required_cols_leverage):
            test_row = ini[required_cols_leverage].dropna().head(1)
            if len(test_row) > 0:
                row = test_row.iloc[0]
                try:
                    result = calculate_leverage_multiple(
                        row['Collat Amount'],
                        row['Collat CCY'],
                        row['Nominal'],
                        row['Currency'],
                        row['Maturity Date']
                    )
                    print(f"  Test sur première ligne valide: '{result}' (type: {type(result).__name__})")
                except Exception as e:
                    print(f"  ✗ ERREUR lors du test: {type(e).__name__}: {str(e)}")
                    import traceback
                    print(f"  Traceback: {traceback.format_exc()}")
            else:
                print("  ⚠ Aucune ligne avec toutes les colonnes nécessaires")
        else:
            print(f"  ✗ Colonnes manquantes: {[col for col in required_cols_leverage if col not in ini.columns]}")
else:
    print("  ✗ Fonction calculate_leverage_multiple non trouvée")

# Test 7: Vérification de la colonne Equiv EUR dans le tableau final
print("\n[TEST 7] Vérification de la colonne Equiv EUR dans le tableau final")
if 'tableau_final' in locals() and not tableau_final.empty:
    if 'Equiv EUR' in tableau_final.columns:
        print("  ✓ Colonne 'Equiv EUR' trouvée")
        non_empty = tableau_final['Equiv EUR'].notna() & (tableau_final['Equiv EUR'] != '')
        print(f"  Nombre de valeurs non vides: {non_empty.sum()} / {len(tableau_final)}")
        if non_empty.sum() > 0:
            sample_values = tableau_final[non_empty]['Equiv EUR'].head(5).tolist()
            print(f"  Exemples de valeurs: {sample_values}")
        else:
            print("  ⚠ ATTENTION: Toutes les valeurs sont vides!")
    else:
        print("  ✗ Colonne 'Equiv EUR' MANQUANTE")
else:
    print("  ✗ Tableau final non trouvé ou vide")

print("\n" + "=" * 80)
print("FIN DES TESTS DE DIAGNOSTIC")
print("=" * 80)
